In [1]:
import os
import tensorflow as tf
from tensorflow.keras import Model
import pathlib

In [4]:
# Load model
# model = tf.keras.models.load_model("faceReader\\xNet_v3.2.0_SGD_128x128_8028_8063")
model = tf.keras.models.load_model("..\\faceReader\\cmu_0.0.1\\")

In [5]:
infer = model.signatures['serving_default']
print(infer.structured_input_signature)
print(infer.structured_outputs)

((), {'input_1': TensorSpec(shape=(None, 80, 80, 3), dtype=tf.float32, name='input_1')})
{'dense_2': TensorSpec(shape=(None, 7), dtype=tf.float32, name='dense_2')}


In [6]:
# converter = tf.lite.TFLiteConverter.from_saved_model("..\\faceReader\\xNet_v4.0.0_SGD_80x80_7779\\")
converter = tf.lite.TFLiteConverter.from_saved_model("..\\faceReader\\cmu_0.0.1\\")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

In [7]:
tflite_model_file = pathlib.Path("..\\tflite_model\\cmu_0.0.1.tflite")
tflite_model_file.write_bytes(tflite_model)

5383664

In [10]:
import tflite_runtime.interpreter as tflite
import numpy as np

In [11]:
interpreter = tflite.Interpreter(model_path="..\\tflite_model\\cmu_0.0.1.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[0.9290683  0.10492983 0.05829364 0.09764606 0.01316643 0.06710875
  0.00992599]]
